# COMP541 - LAB #2
## Classifying MNIST digits with a softmax classifier

In this assignment, you will implement a softmax classifier to predict the digit presented in a given image.
We will use the MNIST dataset for this task. Please first skim through the notebook. Then complete the following steps mentioned in the main function:

1. minibatch
2. init_params
3. forward and backward propagation
    * softmax_forw
    * softmax_back_and_loss
4. grad_check
5. train

Firstly, we install **Knet** and **MLDatasets** only to be able to use MNIST dataset and to import some functions for testing purposes.
Please execute this cell and enter `y` to download the dataset.

In [1]:
using Pkg; for p in ["MLDatasets"]; Pkg.add(p); end
using Printf, Random, Test, Statistics
using MLDatasets: MNIST

@info "Adding MLDatasets"
@test size.(MNIST.traindata(Float32)) == ((28, 28, 60000), (60000,))
@test size.(MNIST.testdata(Float32)) == ((28, 28, 10000), (10000,))

   Updating registry at `C:\Users\volkan\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\volkan\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\volkan\.julia\environments\v1.5\Manifest.toml`
┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1278
┌ Info: Adding MLDatasets
└ @ Main In[1]:5


Test Passed

## Exercise 1

Here you should implement `minibatch` function which takes raw input array `x` and gold labels arrays `y` and splits them into mini batches to be processed.
Hints: you can check arrays size with `size` function and reshape them using `reshape`

In [12]:
"""
    minibatch(x, y, bs=100)

Return a list of minibatches [(xi,yi)...] given data tensors x, y and batchsize.

The last dimension of x and y give the number of instances and should be equal.
"""
function minibatch(x, y, bs=100)
    data = Any[]
    length = size(x, 1);
    final = length / bs; #chunking the dataset
    size_x = size(x)
    size_y = size(y)
    size_x_new = collect(size_x); size_x_new[end] = bs;
    size_y_new = collect(size_y); size_y_new[end] = bs;

    x = reshape(x, :, size_x[end]) # Access only to number of instances without knowing the internal dimension
    y = reshape(y, :, size_y[end]) # Access only to number of instances without knowing the internal dimension
    
        # Minibatching operation with re-reshaping the data into original size
    data = [(reshape(x[:, (k-1)*bs + 1: k*bs],Tuple(size_x_new)), reshape(y[:, (k-1)*bs + 1: k*bs], Tuple(size_y_new))) for k in 1:bs]
    
    return data
end

@info "Testing minibatch function"
@test mean(minibatch(MNIST.testdata(Float32)...)[1][1]) ≈ 0.11988331
@test size.(minibatch(MNIST.testdata(Float32)...)[100]) == ((28, 28, 100), (100,))

┌ Info: Testing minibatch function
└ @ Main In[12]:26


Test Passed

## Exercise 2

Here you should implement `init_params` function which will be used to produce the initial values of the weights.
Hints : look up `randn` and `zeros` functions using `@doc`

In [13]:
"""
    init_params(ninputs, noutputs)

Return a tuple of randomly generated W(sampled from N(0, 1e-3)) and b(must be zeros vector) params of softmax.
"""
function init_params(ninputs::Int, noutputs::Int)
    w = randn(noutputs, ninputs) * 1e-3;
    b = zeros(noutputs,1)
    return w,b
end
Random.seed!(1)
W, b = init_params(150, 100)

@info "Testing init_params function"
@test mean(W) ≈ 2.6869455422978772e-6
@test size(b) == (100, 1)

┌ Info: Testing init_params function
└ @ Main In[13]:14


Test Passed

## Exercise 3.1

This function will takes three arguments, model weights `w`, `b` and a single minibatch input data `x`, applies the affine transformation and softmax function and returns predicted probabilities.
After applying the affine transformation by multplying the input minibatch with the weights and adding the bias vector, you need to implement softmax function as follows:

\begin{eqnarray}{\displaystyle P(y=j\mid \mathbf {x} )={\frac {e^{\mathbf {x} \mathbf {w} _{j} + \mathbf {b} _{j}}}{\sum _{k=1}^{K}e^{\mathbf {x} \mathbf {w} _{j} + \mathbf {b} _{k}}}}}\end{eqnarray}

In [14]:
"""
    softmax_forw(W, b, x)

Return the predicted probabilities of softmax function
"""
function softmax_forw(W, b, x)
    logits = W * x .+ b
    logits .-= maximum(logits);
    probs = exp.(logits)
    probs ./= sum(probs, dims = 1)
    return probs
end

Random.seed!(1)
W, b = init_params(150, 10)

@info "Testing softmax_forw"
@test sum(softmax_forw(W, b, randn(150, 32))[:, 1]) == 1.0

┌ Info: Testing softmax_forw
└ @ Main In[14]:17


Test Passed

## Exercise 3.2
In this function you should firstly do the forward pass using the previous function that you implemented, after that you should calculate negative log likelihood loss:

\begin{eqnarray}{\widehat {\ell \,}}(w, b \,;x)={\frac {-1}{n}}\sum _{i=1}^{n} y_{i}\ln f(x_{i}\mid w,b )\end{eqnarray}
And then you should calculate the gradients of w and b and return them with the loss value.
functions you may use: `log`, `sum`

In [16]:
"""
    softmax_back_and_loss(W, b, x, ygold)

Do softmax forward pass and Return the loss and the gradients of w and b.
"""
function softmax_back_and_loss(W, b, x, ygold)
    probs = softmax_forw(W, b, x);
    class = argmax(ygold, dims = 1)
    loss = sum(-log.(probs[class])) ./ size(ygold, 2)
    
    gradf = probs
    gradf[class] .-= 1
    gradf /= size(ygold, 2)

    gradw = (gradf * x') 
    gradb = vec(sum(gradf, dims=2))
    return loss, gradw, gradb
end

Random.seed!(1)
x = randn(150, 32)
ygold = zeros(10, 32)
ygold[1, :] .= 1
loss, gradw, gradb = softmax_back_and_loss(W, b, x, ygold)

@info "Testing softmax_back_and_loss"
@test loss ≈ 2.302861396444973
@test mean(gradw) ≈ 2.7582103268031233e-19
#@test isapprox(mean(gradw), 2.7582103268031233e-19, atol= 1e-10)
@test mean(gradb) ≈ -1.3877787807814458e-18

Test Failed at In[16]:28
  Expression: mean(gradw) ≈ 2.7582103268031233e-19
   Evaluated: 5.568462357885551e-19 ≈ 2.7582103268031233e-19


┌ Info: Testing softmax_back_and_loss
└ @ Main In[16]:26


LoadError: There was an error during testing

## Exercise 4
Given model weights `W` and `b`, and one minibatch input data `x` and true labels `ygold` as input parameters, this function should display info about whether your gradient calculation procedure passes the gradient check test or not.
Your part of this function is to implement the `numeric_gradient` function, which should calculate the numeric gradients `gw` and `gb` and return them.
Hint: you'll need to do the calculation of the loss for each single value of the parameters twice.
To calculate the numeric gradient of a function recall this:

\begin{eqnarray}f'(x)={\frac {f(x+h)-f(x-h)}{2h}}\end{eqnarray}

In [17]:
"""
    grad_check(W, b, x, ygold)

Check the accuracy of gradients of `W` and `b` which are calculated by `softmax_back_and_loss` function.

This function does that by comparison with the numeric gradients.
"""
function grad_check(W, b, x, ygold)
    """
        numeric_gradient()

    Return numeric gradients of model weights `(gw, gb)`
    """
    function numeric_gradient()
        epsilon = 0.00001

        gw = zeros(size(W)) # gradient of W
        
        gb = zeros(size(b)) # gradient of b
        

        for k in 1:size(W,1)
            b[k] += epsilon # Forward evaluation
            lossplus, _, _ = softmax_back_and_loss(W, b, x, ygold)
            
            b[k] -= 2*epsilon # Backwward evaluation
            lossnegative, _, _ = softmax_back_and_loss(W, b, x, ygold) 

            b[k] += epsilon # Original form
            
            gb[k] = (lossplus - lossnegative) / (2 * epsilon)
            
          for m in 1:size(W,2)
                
                W[k, m] += epsilon # Forward evaluation
                lossplus, _, _ = softmax_back_and_loss(W, b, x, ygold)
                
                W[k, m] -= 2*epsilon # Backward evaluation
                lossnegative, _, _ = softmax_back_and_loss(W, b, x, ygold) 
                
                W[k,m] += epsilon # Original form
                
                gw[k, m] = (lossplus - lossnegative) / (2 * epsilon)


            end
        end
        
        return gw, gb
    end

    _,gradW,gradB = softmax_back_and_loss(W, b, x, ygold)
    
    gw, gb = numeric_gradient()
    diff = sqrt(sum((gradW - gw) .^ 2) + sum((gradB - gb) .^ 2))
    println("Diff: $diff")
    if diff < 1e-7
        println("Gradient Checking Passed")
        return true
    else
        println("Diff must be < 1e-7")
        return false
    end
end

Random.seed!(1)
W, b = init_params(150, 10)
x = randn(150, 32)
ygold = zeros(10, 32)
ygold[1, :] .= 1

@info "Testing grad_check"
@test grad_check(W, b, x, ygold)

┌ Info: Testing grad_check
└ @ Main In[17]:72


Diff: 9.879098353252534e-10
Gradient Checking Passed


Test Passed

## Exercise 5
Given model weights `W` and `b`, set of minibatches `data` and learning rate `lr` as input this function should iterate over
the whole dataset and in each iteration the weights should be updated using the gradients obtained from `softmax_back_and_loss` function call.
Remember the update step of gradient descent optimization algorithm:

\begin{eqnarray}w_{i}:=w_{i}-\eta \nabla w_{i}\end{eqnarray}

In [18]:
"""
    train(W, b, data, lr=0.15)

Update the models weights `W`, `b` using the `data` with learning rate of `lr` and Return the average loss.
"""
function train(W, b, data, lr=0.15)
    totalcost = 0.0
    numins = 0
    for (x, y) in data
        loss, gradw, gradb = softmax_back_and_loss(W, b, x, y)
        
        W .-= lr .* gradw
        b .-= lr .* gradb
        
        totalcost += loss
        numins += 1
    end

    avgcost = totalcost / numins
end

Random.seed!(1)
W, b = init_params(150, 10)
ygold = zeros(10, 32)
ygold[1, :] .= 1

@info "Testing train(W, b, data, lr=0.15) function"
@test train(W, b, [(randn(150, 32), ygold) for i=1:5]) == 2.037075694591626
@test mean(W) == -1.1840149302803448e-5
@test mean(b) == -2.7755575615628915e-18

┌ Info: Testing train(W, b, data, lr=0.15) function
└ @ Main In[18]:27


Test Failed at In[18]:28
  Expression: train(W, b, [(randn(150, 32), ygold) for i = 1:5]) == 2.037075694591626
   Evaluated: 2.0370756945916257 == 2.037075694591626


LoadError: There was an error during testing

Don't touch this cell. Read it carefully.

In [19]:
"""
    accuracy(ygold, ypred)

Return accuracy true labels (ygold) and predicted scores as input for single minibatch.
"""
function accuracy(ygold, ypred)
    correct = 0.0
    for i=1:size(ygold, 2)
        correct += findmax(ygold[:,i]; dims=1)[2] == findmax(ypred[:, i]; dims=1)[2] ? 1.0 : 0.0
    end
    return correct / size(ygold, 2)
end

Random.seed!(1)
W, b = init_params(150, 10)
ygold = zeros(10, 32)

@info "Testing accuracy(ygold, ypred) function"
@test accuracy(ygold, softmax_forw(W, b, randn(150, 32))) == 0.09375

┌ Info: Testing accuracy(ygold, ypred) function
└ @ Main In[19]:18


Test Passed

Don't touch this cell. Read it carefully.

In [20]:
function main()
    Random.seed!(12345)

    # Size of input vector (MNIST images are 28x28)
    ninputs = 28 * 28

    # Number of classes (MNIST images fall into 10 classes)
    noutputs = 10

    #### Data loading & preprocessing
    #
    #  In this section, we load the input and output data,
    #  prepare data to feed into softmax model.
    #  For softmax regression on MNIST pixels,
    #  the input data is the images, and
    #  the output data is the labels.
    #  Size of xtrn: (28,28,60000)
    #  Size of xtrn must be: (784, 60000)
    #  Size of xtst must be: (784, 10000)
    #  Size of ytrn must be: (10, 60000)
    #  Size of ytst must be: (10, 10000)

    xtrn,ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10
    xtst,ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10
    xtrn = reshape(xtrn, 784, 60000)
    xtst = reshape(xtst, 784, 10000)
    function to_onehot(x)
        onehot = zeros(10, 1)
        onehot[x, 1] = 1.0
        return onehot
    end

    ytrn = hcat(map(to_onehot, ytrn)...)
    ytst = hcat(map(to_onehot, ytst)...)

    # STEP 1: Create minibatches
    #   Complete the minibatch function
    #   It takes the input matrix (X) and gold labels (Y)
    #   returns list of tuples contain minibatched input and labels (x, y)
    bs = 100
    trn_data = minibatch(xtrn, ytrn, bs)

    # STEP 2: Initialize parameters
    #   Complete init_params function
    #   It takes number of inputs and number of outputs(number of classes)
    #   It returns randomly generated W matrix and bias vector
    #   Sample from N(0, 0.001)

    W, b = init_params(ninputs, noutputs)

    # STEP 3: Implement softmax_forw and softmax_back_and_loss
    #   softmax_forw function takes W, b, and data
    #   calculates predicted probabilities
    #
    #   softmax_back_and_loss function obtains probabilites by calling
    #   softmax_forw then calculates soft loss and gradients of W and b

    # STEP 4: Gradient checking
    #   Skip this part for the lab session.
    #   As with any learning algorithm, you should always check that your
    #   gradients are correct before learning the parameters.
    debug = true # Turn this parameter off, after gradient checking passed
    if debug
        grad_check(W, b, xtrn[:, 1:100], ytrn[:, 1:100])
    end

    lr = 0.15

    # STEP 5: Training
    #   The train function takes model parameters and the data
    #   Trains the model over minibatches
    #   For each minibatch, first cost and gradients are calculated then model parameters are updated
    #   train function returns the average cost per instance

    for i=1:50
        cost = train(W, b, trn_data, lr)
        pred = softmax_forw(W, b, xtrn)
        trnacc = accuracy(ytrn, pred)
        pred = softmax_forw(W, b, xtst)
        tstacc = accuracy(ytst, pred)
        @printf("epoch: %d softloss: %g trn accuracy: %g tst accuracy: %g\n", i, cost, trnacc, tstacc)
    end

end


main (generic function with 1 method)

In [21]:
main()

#= Example Output
Diff: 1.8292339049184216e-9
Gradient Checking Passed
epoch: 1 softloss: 0.481559 trn accuracy: 0.896983 tst accuracy: 0.9064
epoch: 2 softloss: 0.339105 trn accuracy: 0.907617 tst accuracy: 0.9119
epoch: 3 softloss: 0.31604 trn accuracy: 0.912017 tst accuracy: 0.9142
epoch: 4 softloss: 0.303876 trn accuracy: 0.914783 tst accuracy: 0.9156
epoch: 5 softloss: 0.29597 trn accuracy: 0.916567 tst accuracy: 0.9172
epoch: 6 softloss: 0.290259 trn accuracy: 0.918033 tst accuracy: 0.9187
epoch: 7 softloss: 0.285858 trn accuracy: 0.919233 tst accuracy: 0.9198
epoch: 8 softloss: 0.282317 trn accuracy: 0.920083 tst accuracy: 0.92
epoch: 9 softloss: 0.279378 trn accuracy: 0.9209 tst accuracy: 0.9204
epoch: 10 softloss: 0.276879 trn accuracy: 0.921717 tst accuracy: 0.9211
epoch: 11 softloss: 0.274716 trn accuracy: 0.92225 tst accuracy: 0.9207
epoch: 12 softloss: 0.272816 trn accuracy: 0.92305 tst accuracy: 0.9214
epoch: 13 softloss: 0.271127 trn accuracy: 0.923667 tst accuracy: 0.9214
epoch: 14 softloss: 0.269609 trn accuracy: 0.924133 tst accuracy: 0.9215
epoch: 15 softloss: 0.268235 trn accuracy: 0.924417 tst accuracy: 0.922
epoch: 16 softloss: 0.26698 trn accuracy: 0.9247 tst accuracy: 0.9219
epoch: 17 softloss: 0.265828 trn accuracy: 0.924933 tst accuracy: 0.9218
epoch: 18 softloss: 0.264764 trn accuracy: 0.92505 tst accuracy: 0.922
epoch: 19 softloss: 0.263777 trn accuracy: 0.925367 tst accuracy: 0.9223
epoch: 20 softloss: 0.262856 trn accuracy: 0.92575 tst accuracy: 0.9225
epoch: 21 softloss: 0.261995 trn accuracy: 0.9263 tst accuracy: 0.9227
epoch: 22 softloss: 0.261186 trn accuracy: 0.926567 tst accuracy: 0.9226
epoch: 23 softloss: 0.260424 trn accuracy: 0.9269 tst accuracy: 0.9229
epoch: 24 softloss: 0.259704 trn accuracy: 0.92715 tst accuracy: 0.9227
epoch: 25 softloss: 0.259022 trn accuracy: 0.927367 tst accuracy: 0.9227
epoch: 26 softloss: 0.258374 trn accuracy: 0.9275 tst accuracy: 0.9229
epoch: 27 softloss: 0.257758 trn accuracy: 0.927767 tst accuracy: 0.923
epoch: 28 softloss: 0.257171 trn accuracy: 0.928083 tst accuracy: 0.9229
epoch: 29 softloss: 0.25661 trn accuracy: 0.92825 tst accuracy: 0.9231
epoch: 30 softloss: 0.256073 trn accuracy: 0.92835 tst accuracy: 0.9229
epoch: 31 softloss: 0.255558 trn accuracy: 0.928517 tst accuracy: 0.923
epoch: 32 softloss: 0.255064 trn accuracy: 0.928783 tst accuracy: 0.9228
epoch: 33 softloss: 0.254589 trn accuracy: 0.92895 tst accuracy: 0.9229
epoch: 34 softloss: 0.254133 trn accuracy: 0.9291 tst accuracy: 0.9227
epoch: 35 softloss: 0.253692 trn accuracy: 0.929167 tst accuracy: 0.9228
epoch: 36 softloss: 0.253268 trn accuracy: 0.92925 tst accuracy: 0.9227
epoch: 37 softloss: 0.252858 trn accuracy: 0.929417 tst accuracy: 0.923
epoch: 38 softloss: 0.252462 trn accuracy: 0.929567 tst accuracy: 0.9229
epoch: 39 softloss: 0.252078 trn accuracy: 0.929667 tst accuracy: 0.9228
epoch: 40 softloss: 0.251707 trn accuracy: 0.929783 tst accuracy: 0.9229
epoch: 41 softloss: 0.251347 trn accuracy: 0.929867 tst accuracy: 0.9231
epoch: 42 softloss: 0.250998 trn accuracy: 0.930067 tst accuracy: 0.9235
epoch: 43 softloss: 0.25066 trn accuracy: 0.9301 tst accuracy: 0.9235
epoch: 44 softloss: 0.250331 trn accuracy: 0.930233 tst accuracy: 0.9235
epoch: 45 softloss: 0.250011 trn accuracy: 0.930333 tst accuracy: 0.9235
epoch: 46 softloss: 0.2497 trn accuracy: 0.9305 tst accuracy: 0.9237
epoch: 47 softloss: 0.249397 trn accuracy: 0.930583 tst accuracy: 0.9238
epoch: 48 softloss: 0.249102 trn accuracy: 0.9307 tst accuracy: 0.9239
epoch: 49 softloss: 0.248815 trn accuracy: 0.93085 tst accuracy: 0.9242
epoch: 50 softloss: 0.248535 trn accuracy: 0.930933 tst accuracy: 0.9243
=#

Diff: 9.390935718438783e-10
Gradient Checking Passed
epoch: 1 softloss: 0.823248 trn accuracy: 0.859417 tst accuracy: 0.8667
epoch: 2 softloss: 0.462602 trn accuracy: 0.877583 tst accuracy: 0.8843
epoch: 3 softloss: 0.40045 trn accuracy: 0.885583 tst accuracy: 0.8914
epoch: 4 softloss: 0.368761 trn accuracy: 0.89005 tst accuracy: 0.8973
epoch: 5 softloss: 0.348327 trn accuracy: 0.893717 tst accuracy: 0.9002
epoch: 6 softloss: 0.333549 trn accuracy: 0.896367 tst accuracy: 0.9027
epoch: 7 softloss: 0.322106 trn accuracy: 0.89815 tst accuracy: 0.9049
epoch: 8 softloss: 0.312833 trn accuracy: 0.8998 tst accuracy: 0.9066
epoch: 9 softloss: 0.305072 trn accuracy: 0.9014 tst accuracy: 0.9078
epoch: 10 softloss: 0.298421 trn accuracy: 0.90295 tst accuracy: 0.9077
epoch: 11 softloss: 0.292614 trn accuracy: 0.903833 tst accuracy: 0.9083
epoch: 12 softloss: 0.28747 trn accuracy: 0.9046 tst accuracy: 0.9083
epoch: 13 softloss: 0.282858 trn accuracy: 0.905233 tst accuracy: 0.9085
epoch: 14 softloss

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*